In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True , as_supervised = True)

mnist_train,mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = 0.1*mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

buffer_size = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

batch_size = 100

train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs , validation_targets = next(iter(validation_data))


## MODEL

In [3]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                              tf.keras.layers.Flatten(input_shape = (28,28,1)),
                              tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                              tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                              tf.keras.layers.Dense(output_size,activation='softmax')
                            ])

## compiling with optimizer and loss 

In [4]:
model.compile(optimizer= 'adam', loss= 'sparse_categorical_crossentropy', metrics= ['accuracy'])

## Training

In [5]:
num_epochs = 5
model.fit(train_data, epochs = num_epochs, validation_data= (validation_inputs,validation_targets), verbose = 2)

Epoch 1/5


540/540 - 3s - loss: 0.3288 - accuracy: 0.9048 - val_loss: 0.1650 - val_accuracy: 0.9548
Epoch 2/5
540/540 - 2s - loss: 0.1382 - accuracy: 0.9587 - val_loss: 0.1239 - val_accuracy: 0.9663
Epoch 3/5
540/540 - 2s - loss: 0.0987 - accuracy: 0.9704 - val_loss: 0.1001 - val_accuracy: 0.9733
Epoch 4/5
540/540 - 2s - loss: 0.0752 - accuracy: 0.9779 - val_loss: 0.0893 - val_accuracy: 0.9748
Epoch 5/5
540/540 - 2s - loss: 0.0601 - accuracy: 0.9824 - val_loss: 0.0656 - val_accuracy: 0.9827


## Testing

In [6]:
test_loss, test_accuracy = model.evaluate(test_data, verbose= 2)

10/10 - 0s - loss: 0.0813 - accuracy: 0.9758
